Problema de negócio: Uma grande multinacional varejista do ramo de supermercados deseja ingressar no mercado brasileiro

    - Dizer ao cliente o porque ele deve entrar nesse mercado utilizando esses grupos.
    - Depois da realização da separação por grupos, como ainda vão ter muitos disponíveis, aplicar novamente a metodologia de análise para ter um número satisfatório.

- Realize uma classificação dos municípios brasileiros com base nas informações disponíveis e faça uma caracterização dos municípios em grupos.
    - Separar os munícipos em grupos: 


- Elabore uma forma de classificar um novo município entre os grupos já criados na etapa anterior!


- Responda em sua apresentação quais grupos de municípios deveriam ser a porta de entrada para empresa no país e porque!

Métricas de clusterização:

    - WSS (within sum square)
    - SS ( Silhoute Score ) 

# 0.0 Imports

In [2]:

import pandas as pd
from pandas_profiling import ProfileReport 
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from plotly import express as px

from IPython.core.display import HTML

from sklearn import cluster as c
from sklearn import metrics as m
from sklearn import preprocessing as pp
from sklearn import decomposition as dd

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import umap.umap_ as umap


D:\Anaconda\envs\plusoft\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'umap.umap_'

## 0.1 Helper Functions

In [ ]:
def jupyter_settings():
  %matplotlib inline
  %pylab inline
  plt.style.use( 'bmh' )
  plt.rcParams['figure.figsize'] = [25, 12]
  plt.rcParams['font.size'] = 24
  display( HTML( '<style>.container { width:100% !important; }</style>') )
  pd.options.display.max_columns = None
  pd.options.display.max_rows = None
  pd.set_option( 'display.expand_frame_repr', False )
  sns.set()

In [ ]:
jupyter_settings()

## 0.2 Load Dataset

In [ ]:
df_raw = pd.read_excel('case_plusoft01.xls')
df_raw.head()

In [ ]:
df_raw.head()

# 1.0 Descrição dos Dados

In [ ]:
df1 = df_raw.copy()

## 1.1 Dimensão dos dados

In [ ]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )

## 1.2 Tipo dos dados

In [ ]:
df1.dtypes

## 1.3 Check NA

In [ ]:
# verificação na tabela de dados NAN
df1.isna().sum()

## 1.4 Estatística Descritiva

### 1.4.1 Atributos Numéricos

- Na primeira rodada, não realizo a estatistica descritiva

In [ ]:
df1 = df1.select_dtypes(exclude = ['object'])
df1.head()

In [ ]:
df1.columns

In [ ]:
# tendência central: Média e Mediana
ct1 = pd.DataFrame(df1.apply(np.mean)).T
ct2 = pd.DataFrame(df1.apply(np.median) ).T

In [ ]:
# dispersão: std, min, max, range, skew, kurtosis

d1 = pd.DataFrame(df1.apply(np.std) ).T
d2 = pd.DataFrame(df1.apply(min ) ).T
d3 = pd.DataFrame(df1.apply(max ) ).T
d4 = pd.DataFrame(df1.apply(lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame(df1.apply(lambda x: x.skew() )).T
d6 = pd.DataFrame(df1.apply(lambda x: x.kurtosis() )).T

In [ ]:
m = pd.concat( [d2, d3, d4, ct1,ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std',
             'skew', 'kurtosis']

In [ ]:
m

# 2.0 Feature Engineering


## 2.1 Criação das features

In [ ]:
df2 = df1.copy()


In [ ]:
##### features populacionais ######
 

df2['cresc_popu_1991_2000'] = ( df2['População total, 2000'] - df2['População total, 1991']) / df2['População total, 1991']


#### features financeiras

## Alimentos in natura: 7,29% + Alimentos industrializados: 12,40% + Sabões e artigos de limpeza: 2,12 = 21,81%
df2['receita_pop_mercado_2000']  = (df2['População total, 2000'] * df2['Renda per Capita, 2000'] ) * 0.22




In [ ]:
df2.head()

# 3.0 Filtragem das Variáveis

In [ ]:
df3 = df2.copy()
df3.head()

# 4.0 Análise Exploratória dos Dados (EDA)

In [ ]:
df4 = df3.copy()
print( 'Number of Cols: {}'.format( df4.shape[1] ) )

## 4.1 Análise Univariada


In [ ]:
profile = ProfileReport(df4 )
profile.to_file ( 'output.html' )

In [ ]:
df4.dtypes

In [ ]:
#features de área
df4['Área (km²)']
df4['Distância à capital (km)']

df4[['Área (km²)','Distância à capital (km)']].hist(bins = 25);

In [ ]:
#features de natalidade
df4['Esperança de vida ao nascer, 2000']
df4['Mortalidade até um ano de idade, 2000']
df4['Taxa de fecundidade total, 2000']

df4[['Esperança de vida ao nascer, 2000','Mortalidade até um ano de idade, 2000',
     'Taxa de fecundidade total, 2000'  ]].hist();

In [ ]:
#features de alfabetização
df4['Percentual de pessoas de 25 anos ou mais analfabetas, 2000']
df4['Taxa bruta de freqüência à escola, 2000']
df4['Taxa de alfabetização, 2000']
df4['Média de anos de estudo das pessoas de 25 anos ou mais de idade, 2000']

df4[['Percentual de pessoas de 25 anos ou mais analfabetas, 2000',
     'Taxa bruta de freqüência à escola, 2000',
     'Taxa de alfabetização, 2000',
     'Média de anos de estudo das pessoas de 25 anos ou mais de idade, 2000'     
    ]].hist();

In [ ]:
#features de população
df4['População de 25 anos ou mais de idade, 1991']
df4['População de 25 anos ou mais de idade, 2000']
df4['População de 65 anos ou mais de idade, 1991']
df4['População de 65 anos ou mais de idade, 2000']
df4['População total, 1991']
df4['População total, 2000']
df4['População urbana, 2000']
df4['População rural, 2000']
df4['cresc_popu_1991_2000']

df4[['População de 25 anos ou mais de idade, 1991',
    'População de 25 anos ou mais de idade, 2000',
     'População de 65 anos ou mais de idade, 1991',
     'População de 65 anos ou mais de idade, 2000',
     'População total, 1991',
     'População total, 2000',
     'População urbana, 2000',
     'População rural, 2000',
     'cresc_popu_1991_2000' 
     ]].hist(bins = 25);

In [ ]:
#features de desenvolvimento socio-econômico
df4['Renda per Capita, 2000']
df4['Índice de Gini, 2000']
df4['Intensidade da indigência, 2000']
df4['Intensidade da pobreza, 2000']
df4['Índice de Desenvolvimento Humano Municipal, 2000']

df4[['Índice de Gini, 2000',
     'Intensidade da indigência, 2000','Intensidade da pobreza, 2000',
     'Índice de Desenvolvimento Humano Municipal, 2000'
    ]].hist(bins = 25);

## 4.2 Análise Bivariada

Análise da população total em relação a renda per capita

In [ ]:
plt.subplot( 1, 2, 1 )
aux1 = df4[['População total, 2000', 'Renda per Capita, 2000']].groupby( 'População total, 2000' ).sum().reset_index()
sns.scatterplot( x ='População total, 2000', y='Renda per Capita, 2000', data=aux1 );


plt.subplot( 1, 2, 2 )
bins = list( np.arange( 0, 500000, 50000) )
aux1['populacao_total_binned'] = pd.cut( aux1['População total, 2000'],bins=bins )
aux2 = aux1[['populacao_total_binned', 'Renda per Capita, 2000']].groupby('populacao_total_binned').sum().reset_index()
sns.barplot( x='populacao_total_binned', y='Renda per Capita, 2000', data=aux2 );
plt.xticks( rotation=90 );

## 4.3 Análise Multivariada

## 4.4 Análise do Espaço

In [ ]:
df43 = df4.drop(columns = ['Código']).copy()

In [ ]:
mm = pp.MinMaxScaler()

   
df43['lat'] = mm.fit_transform(df43[['lat']])   
df43['long'] = mm.fit_transform(df43[['long']])   
df43['Área (km²)'] = mm.fit_transform(df43[['Área (km²)']])   
df43['Densidade demográfica, 2000'] = mm.fit_transform(df43[['Densidade demográfica, 2000']])   
df43['Distância à capital (km)'] = mm.fit_transform(df43[['Distância à capital (km)']])   
df43['Esperança de vida ao nascer, 2000'] = mm.fit_transform(df43[['Esperança de vida ao nascer, 2000']])   
df43['Mortalidade até um ano de idade, 2000'] = mm.fit_transform(df43[['Mortalidade até um ano de idade, 2000']])   
df43['Taxa de fecundidade total, 2000'] = mm.fit_transform(df43[['Taxa de fecundidade total, 2000']])   
df43['Percentual de pessoas de 25 anos ou mais analfabetas, 2000'] = mm.fit_transform(df43[['Percentual de pessoas de 25 anos ou mais analfabetas, 2000']])   
df43['Renda per Capita, 2000'] = mm.fit_transform(df43[['Renda per Capita, 2000']])   
df43['Índice de Gini, 2000'] = mm.fit_transform(df43[['Índice de Gini, 2000']])   
df43['Intensidade da indigência, 2000'] = mm.fit_transform(df43[['Intensidade da indigência, 2000']])   
df43['Intensidade da pobreza, 2000'] = mm.fit_transform(df43[['Intensidade da pobreza, 2000']])   
df43['Índice de Desenvolvimento Humano Municipal, 2000'] = mm.fit_transform(df43[['Índice de Desenvolvimento Humano Municipal, 2000']])   
df43['Taxa bruta de freqüência à escola, 2000'] = mm.fit_transform(df43[['Taxa bruta de freqüência à escola, 2000']])   
df43['Taxa de alfabetização, 2000'] = mm.fit_transform(df43[['Taxa de alfabetização, 2000']])   
df43['Média de anos de estudo das pessoas de 25 anos ou mais de idade, 2000'] = mm.fit_transform(df43[['Média de anos de estudo das pessoas de 25 anos ou mais de idade, 2000']])   
df43['População de 25 anos ou mais de idade, 1991'] = mm.fit_transform(df43[['População de 25 anos ou mais de idade, 1991']])   
df43['População de 25 anos ou mais de idade, 2000'] = mm.fit_transform(df43[['População de 25 anos ou mais de idade, 2000']])   
df43['População de 65 anos ou mais de idade, 1991'] = mm.fit_transform(df43[['População de 65 anos ou mais de idade, 1991']])   
df43['População de 65 anos ou mais de idade, 2000'] = mm.fit_transform(df43[['População de 65 anos ou mais de idade, 2000']])   
df43['População total, 1991'] = mm.fit_transform(df43[['População total, 1991']])   
df43['População total, 2000'] = mm.fit_transform(df43[['População total, 2000']])   
df43['População urbana, 2000'] = mm.fit_transform(df43[['População urbana, 2000']])   
df43['População rural, 2000'] = mm.fit_transform(df43[['População rural, 2000']])   
df43['cresc_popu_1991_2000'] = mm.fit_transform(df43[['cresc_popu_1991_2000']])   
df43['receita_pop_mercado_2000']= mm.fit_transform(df43[['receita_pop_mercado_2000']]) 

X = df43.copy()

### 4.3.1 PCA

In [ ]:
pca = dd.PCA(n_components = X.shape[1], )

principal_components = pca.fit_transform( X )

# plot explained variable
features = range( pca.n_components_ )

plt.bar(features, pca.explained_variance_ratio_, color = 'black')

#pca component
df_pca = pd.DataFrame (principal_components)

In [ ]:
sns.scatterplot (x = 0, y = 1, data = df_pca) 

In [ ]:
df_pca.head()

# 5.0 Data Preparation

In [ ]:
df5 = df4.copy()

In [ ]:
## Standard Scaler
ss = pp.StandardScaler()

df5['receita_pop_mercado_2000'] = ss.fit_transform( df5[['receita_pop_mercado_2000']] )

In [ ]:
df5.head()

# 6.0 Feature Selection

In [ ]:
df6 = df5.copy()
df6.head()

# 7.0 Hyperparameter Fine-Tunning

In [ ]:
# tiro a coluna código para aplicar o kluster somente nas features relevantes
x = df6.drop( columns=['Código', 'nome' ] )
x.head()

In [ ]:
# criação dos cluster para avaliar a melhor métrica
clusters = [2, 3, 4, 5, 6]

## 7.1 Within-Cluster Sum of Square (WSS)

In [ ]:
wss = []

for k in clusters:
    # model definition
    kmeans = c.KMeans (init = 'random', n_clusters = k, n_init = 10, max_iter = 300, random_state = 42)
    
    # model training
    kmeans.fit(x)
    
    # validation
    wss.append( kmeans.inertia_ )
    
# plot wss - elbow method
#plt.plot( clusters, wss, linestyle = '--', marker = 'o', color = 'b' )
#plt.xlabel ( 'K')
#plt.ylabel ('Within-Cluster Sum of Square')
#plt.title('WSS vs K' )

In [ ]:
#pontos de inflexão: 3, 4 e 5

kmeans = KElbowVisualizer( c.KMeans(), k= clusters, timings = False  )
kmeans.fit(x)
kmeans.show()

In [ ]:
wss

## 7.2 Sillhouette Score

In [ ]:
kmeans = KElbowVisualizer( c.KMeans(), k= clusters, metric = 'silhouette', timings = False  )
kmeans.fit(x)
kmeans.show()

## 7.3 Sihlouette Analysis

In [ ]:
x.head()

In [ ]:
fig, ax = plt.subplots(3 , 2, figsize = (25,18) )

k = [2, 3, 4, 5, 6]
for k in clusters:    
    km = c.KMeans( n_clusters = k, init = 'random', n_init = 10, max_iter = 100, random_state = 42)
    q, mod = divmod(k, 2)
    visualizer = SilhouetteVisualizer ( km, color = 'yellowbrick', ax = ax[q-1][mod])
    visualizer.fit(x)
    visualizer.finalize()

# 8.0 Model Training

## 8.1 K-Means

In [ ]:
# model definition
k = 6 
kmeans = c.KMeans (init = 'random', n_clusters = k,n_init = 10, max_iter = 300, random_state = 42)

# model training
kmeans.fit( x )

#clustering
labels = kmeans.labels_

## 8.2 Cluster Validation

In [ ]:
## WSS (Within-cluster sum of square)
print('WSS vale: {}'.format( kmeans.inertia_) )

## SS (Silhouette Score)
print('SS vale: {}'.format( m.silhouette_score(x, labels, metric = 'euclidean') ) )

# 9.0 Cluster Analysis

In [ ]:
df9 = df6.copy()
df9['cluster'] = labels
df9.head()

## 9.1 Visualization Inspection

In [ ]:
visualizer = SilhouetteVisualizer(kmeans, colors ='yellowbrick')
visualizer.fit(x)
visualizer.finalize()

## 9.2 2d plot

In [ ]:
df9.head()

In [ ]:
df_viz = df9.drop(columns= 'Código', axis = 1)

In [ ]:
# dificil visualização quando tenho muitas features
sns.pairplot(df_viz, hue = 'cluster')

## 9.3 UMAP 

Manifold: aprendizado por topologia
    
    - visualização da projeção dos dados em alta dimensionalidade

In [ ]:
x.head()

In [ ]:
#n_neighbors: quanto menor, mais perto eu estou vendo, quanto maior, mais de longe eu estou vendo
reducer = umap.UMAP( n_neighbors= 10,random_state = 42)
embedding = reducer.fit_transform(x)

# embedding 
df_viz['embedding_x'] = embedding[:,0]
df_viz['embedding_y'] = embedding[:,1]

#plot UMAP
sns.scatterplot(x = 'embedding_x', y = 'embedding_y',
               hue= 'cluster',
               palette = sns.color_palette('hls', n_colors = len(df_viz['cluster'].unique() ) ), 
               data = df_viz )


## 9.2 Cluster Profile

In [ ]:
df9.head()

In [ ]:
# numero de municipios por cluster
df_cluster1 = df9[['População urbana, 2000', 'cluster']].groupby('cluster').count().reset_index()

# populacao total por cluster
df_cluster2 = df9[['População total, 2000', 'cluster']].groupby('cluster').sum().reset_index()

# porcentagem da populacao por cluster
df_cluster2['perc_populacao'] = 100 * ( df_cluster2['População total, 2000'] /
                                        df_cluster2['População total, 2000'].sum() )                            

# receita total da populacao
df_cluster3 = df9[['receita_pop_mercado_2000', 'cluster']].groupby('cluster').sum().reset_index()

# IDHM
df_cluster4 = df9[['Índice de Desenvolvimento Humano Municipal, 2000',
                   'cluster']].groupby('cluster').mean().reset_index()

# merge dos dataframes
df_cluster = pd.merge( df_cluster1, df_cluster2,how = 'left', on = 'cluster'  )
df_cluster = pd.merge( df_cluster, df_cluster3, how = 'left', on = 'cluster'  )
df_cluster = pd.merge( df_cluster, df_cluster4, how = 'left', on = 'cluster'  )

df_cluster.columns = ['cluster', 'numero_de_municipios', 'População total, 2000', 
                     'perc_populacao, 2000', 'receita_pop_mercado_2000', 'IDHM_medio']


# dataframe final
df_cluster

### Cluster 00:
        
        - Número de municipios: 67
        - População Total, 2000: 18.691.596 (11% da população total)
        - Receita da população destinada a itens de mercado, 2000: R$ 1.365.543.000,000 
        - IDHM Médio, 2000: 0.8


### Cluster 01:
        
        - Número de municipios: 5135
        - População Total, 2000: 72.808.636 (42,8% da população total)
        - Receita da população destinada a itens de mercado, 2000: R$ 2.562.848.000,000 
        - IDHM Médio, 2000: 0.69


### Cluster 02:

        - Número de municipios: 30
        - População Total, 2000: 19.118.581 ( 11,26% da população total )
        - Receita da população destinada a itens de mercado, 2000: R$ 1.631.235.000,000
        - IDHM Médio, 2000: 0.81


### Cluster 03:

        - Número de municipios: 9
        - População Total, 2000: 15.307.394 ( 9,01% da população total)
        - Receita da população destinada a itens de mercado, 2000: R$ 1.680.808.000,00
        - IDHM Médio, 2000: 0.83


### Cluster 04:

        - Número de municipios: 264
        - População Total, 2000: 27.580.807 (16,24% da população total)
        - Receita da população destinada a itens de mercado, 2000: R$ 1.714.149.000,00
        - IDHM Médio, 2000: 0.78


### Cluster 05:

        - Número de municipios: 2
        - População Total, 2000: 16.292.156 ( 9.59% da população total)
        - Receita da população destinada a itens de mercado, 2000: R$ 2.169.294.000,00
        - IDHM Médio, 2000: 0.84
        
        